In [1]:
from typing import Annotated,Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage



In [2]:
llm = ChatOpenAI(model='gpt-4.1-mini',temperature=0)

class State(TypedDict):
    messages: Annotated[list,add_messages]
    summary: str

def ask_llm(state: State):
    summary = state.get("summary","")
    if summary:
        system_message = f"summary of conversation earlier: {summary}"
        messages = [SystemMessage(context=system_message)]+state["messages"]
    else:
        messages = state["messages"]
    
    response = llm.invoke(messages)
    return {"messages":[response]}


def should_continue(state:State)->Literal["summarization", END]:
    messages = state.get("messages","")
    if len(messages) > 6 :
        return "summarization"
    return END


def summarization(state : State ):
    summary = state.get("summary")
    
    if summary:
        # 이전 요약 정보가 있다면 요약 메시지 생성
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above in Korean:"
        )
    else:
        # 요약 메시지 생성
        summary_message = "Create a summary of the conversation above in Korean:"

    messages_and_summary = state["messages"] + [HumanMessage(content=summary_message)]
    total_summary = llm.invoke(messages_and_summary)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary":total_summary.content, "messages":delete_messages}


state_graph = StateGraph(State)
state_graph.add_node('ask_llm',ask_llm)
state_graph.add_node('summarization',summarization)

state_graph.add_edge(START,'ask_llm')
state_graph.add_conditional_edges(
    source='ask_llm',
    path=should_continue
)
state_graph.add_edge('summarization',END)

graph = state_graph.compile()


In [3]:
# 메모리 저장소 생성
memory = MemorySaver()
graph = state_graph.compile(checkpointer=memory)
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	ask_llm(ask_llm)
	summarization(summarization)
	__end__([<p>__end__</p>]):::last
	__start__ --> ask_llm;
	ask_llm -.-> __end__;
	ask_llm -.-> summarization;
	summarization --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [4]:

# query = "2025년 10월 초 서울 근교 데이터 코스 찾아줘"

# graph.invoke({"messages":query})

In [5]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=6,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "0012"},  # 스레드 ID 설정
)

In [6]:
query = "안녕 나는 김자윤이야 ?"

for event in graph.stream({"messages":query},config=config,stream_mode="values"):
    # messages = event["messages"]
    # print(len(messages))
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕 나는 김자윤이야 ?')]
[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?')]


In [7]:
query = "이번주 여행 스케줄 짜고싶어"

for event in graph.stream({"messages":query},config=config,stream_mode="values"):
    # messages = event["messages"]
    # print(len(messages))
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?'), ('human', '이번주 여행 스케줄 짜고싶어')]
[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?'), ('human', '이번주 여행 스케줄 짜고싶어'), ('ai', '좋아요, 김자윤님! 이번 주 여행 스케줄을 함께 짜볼게요. 어디로 여행 가실 예정인가요? 그리고 여행 기간, 선호하는 활동이나 장소, 예산 같은 정보도 알려주시면 더 도움이 될 것 같아요!')]


In [8]:
query = "설악산으로 2박 3일 갈 예정이야"

for event in graph.stream({"messages":query},config=config,stream_mode="values"):
    # messages = event["messages"]
    # print(len(messages))
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?'), ('human', '이번주 여행 스케줄 짜고싶어'), ('ai', '좋아요, 김자윤님! 이번 주 여행 스케줄을 함께 짜볼게요. 어디로 여행 가실 예정인가요? 그리고 여행 기간, 선호하는 활동이나 장소, 예산 같은 정보도 알려주시면 더 도움이 될 것 같아요!'), ('human', '설악산으로 2박 3일 갈 예정이야')]
[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?'), ('human', '이번주 여행 스케줄 짜고싶어'), ('ai', '좋아요, 김자윤님! 이번 주 여행 스케줄을 함께 짜볼게요. 어디로 여행 가실 예정인가요? 그리고 여행 기간, 선호하는 활동이나 장소, 예산 같은 정보도 알려주시면 더 도움이 될 것 같아요!'), ('human', '설악산으로 2박 3일 갈 예정이야'), ('ai', '설악산 2박 3일 여행, 정말 멋지네요! 김자윤님께 맞는 스케줄을 제안해드릴게요.\n\n---\n\n### 1일차  \n- **오전:** 출발 및 설악산 도착  \n- **점심:** 속초 시내에서 간단히 식사 (예: 닭강정, 해물칼국수)  \n- **오후:** 설악산 국립공원 입장 후 권금성 케이블카 타기  \n- **저녁:** 숙소 체크인 및 휴식, 근처 맛집 탐방  \n\n### 2일차  \n- **아침:** 일찍 출발, 대청봉 등반 (체력에 따라 코스 조절 가능)  \n- **점심:** 산에서 간단한 도시락 또는 산장 식사  \n- **오후:** 울산바위 트레킹 또는 비선대 산책  \n- **저녁:** 숙소 복귀 후 온천이나 마사지로 피로 풀기  \n\n### 3일차  \n- **아침:** 설악해변 산책 또는 속초 중앙시장 방문  \n- **점심:** 속초에서 해산물 식사 (회, 생선구이 등)  \n- **오후:** 귀가  \n\n---\n\n추가로 원하시는 활동

In [9]:
values = graph.get_state(config).values
values

{'messages': [HumanMessage(content='안녕 나는 김자윤이야 ?', additional_kwargs={}, response_metadata={}, id='4c153b01-4b17-454f-92f5-adaa7ffda707'),
  AIMessage(content='안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 16, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CIxVaNpwxqIeE4SNLD1YxxTez9T2x', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6bf37c6f-4a44-4fb3-8335-f50031ab8c82-0', usage_metadata={'input_tokens': 16, 'output_tokens': 20, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='이번주

In [10]:
query = "비가오면어떻해야해?"

for event in graph.stream({"messages":query},config=config,stream_mode="values"):
    # messages = event["messages"]
    # print(len(messages))
    print([(message.type, message.content) for message in event["messages"]])


[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?'), ('human', '이번주 여행 스케줄 짜고싶어'), ('ai', '좋아요, 김자윤님! 이번 주 여행 스케줄을 함께 짜볼게요. 어디로 여행 가실 예정인가요? 그리고 여행 기간, 선호하는 활동이나 장소, 예산 같은 정보도 알려주시면 더 도움이 될 것 같아요!'), ('human', '설악산으로 2박 3일 갈 예정이야'), ('ai', '설악산 2박 3일 여행, 정말 멋지네요! 김자윤님께 맞는 스케줄을 제안해드릴게요.\n\n---\n\n### 1일차  \n- **오전:** 출발 및 설악산 도착  \n- **점심:** 속초 시내에서 간단히 식사 (예: 닭강정, 해물칼국수)  \n- **오후:** 설악산 국립공원 입장 후 권금성 케이블카 타기  \n- **저녁:** 숙소 체크인 및 휴식, 근처 맛집 탐방  \n\n### 2일차  \n- **아침:** 일찍 출발, 대청봉 등반 (체력에 따라 코스 조절 가능)  \n- **점심:** 산에서 간단한 도시락 또는 산장 식사  \n- **오후:** 울산바위 트레킹 또는 비선대 산책  \n- **저녁:** 숙소 복귀 후 온천이나 마사지로 피로 풀기  \n\n### 3일차  \n- **아침:** 설악해변 산책 또는 속초 중앙시장 방문  \n- **점심:** 속초에서 해산물 식사 (회, 생선구이 등)  \n- **오후:** 귀가  \n\n---\n\n추가로 원하시는 활동이나 숙소 스타일, 이동 수단 알려주시면 더 맞춤형으로 도와드릴게요!'), ('human', '비가오면어떻해야해?')]
[('human', '안녕 나는 김자윤이야 ?'), ('ai', '안녕, 김자윤! 만나서 반가워요. 어떻게 도와줄까요?'), ('human', '이번주 여행 스케줄 짜고싶어'), ('ai', '좋아요, 김자윤님! 이번 주 여행 스케줄을 함께 짜볼게요. 어디로 여행 가실 예정인가요? 그리고 여행 기간, 선호하는 활동이나 장소, 

In [11]:
values = graph.get_state(config).values
values

{'messages': [HumanMessage(content='비가오면어떻해야해?', additional_kwargs={}, response_metadata={}, id='cc9e4569-07f9-46a2-b0bc-f6bb3e9285ba'),
  AIMessage(content='비 오는 날에도 설악산 여행을 즐길 수 있도록 몇 가지 팁과 대안을 알려드릴게요!\n\n---\n\n### 비 오는 날 설악산 여행 팁\n\n1. **방수 장비 준비**  \n   - 방수 자켓, 우산, 방수 신발 꼭 챙기세요.  \n   - 배낭도 방수 커버로 보호하면 좋아요.\n\n2. **안전 우선!**  \n   - 비가 많이 오면 산길이 미끄럽고 위험할 수 있으니 무리한 등산은 피하세요.  \n   - 산행 전 기상 상황을 꼭 확인하세요.\n\n3. **대체 일정 추천**  \n   - **설악 워터피아**: 속초에 있는 온천 테마파크로 비 오는 날 실내에서 즐기기 좋아요.  \n   - **속초 중앙시장**: 다양한 해산물과 먹거리를 즐기며 시장 구경하기.  \n   - **아바이마을 방문**: 속초의 전통 마을로 실내 카페나 식당에서 휴식 가능.  \n   - **박물관, 갤러리 방문**: 속초 주변에 있는 박물관이나 갤러리 탐방.\n\n4. **숙소에서 휴식**  \n   - 좋은 숙소를 예약했다면, 휴식과 함께 독서나 영화 감상 등 여유로운 시간을 보내는 것도 좋아요.\n\n---\n\n비 오는 날에도 안전하고 즐거운 여행 되시길 바랄게요! 필요하면 비 오는 날 맞춤 일정도 더 자세히 짜드릴 수 있어요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 350, 'prompt_tokens': 460, 'total_tokens': 810, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 

In [12]:
messages = values["messages"]
messages

[HumanMessage(content='비가오면어떻해야해?', additional_kwargs={}, response_metadata={}, id='cc9e4569-07f9-46a2-b0bc-f6bb3e9285ba'),
 AIMessage(content='비 오는 날에도 설악산 여행을 즐길 수 있도록 몇 가지 팁과 대안을 알려드릴게요!\n\n---\n\n### 비 오는 날 설악산 여행 팁\n\n1. **방수 장비 준비**  \n   - 방수 자켓, 우산, 방수 신발 꼭 챙기세요.  \n   - 배낭도 방수 커버로 보호하면 좋아요.\n\n2. **안전 우선!**  \n   - 비가 많이 오면 산길이 미끄럽고 위험할 수 있으니 무리한 등산은 피하세요.  \n   - 산행 전 기상 상황을 꼭 확인하세요.\n\n3. **대체 일정 추천**  \n   - **설악 워터피아**: 속초에 있는 온천 테마파크로 비 오는 날 실내에서 즐기기 좋아요.  \n   - **속초 중앙시장**: 다양한 해산물과 먹거리를 즐기며 시장 구경하기.  \n   - **아바이마을 방문**: 속초의 전통 마을로 실내 카페나 식당에서 휴식 가능.  \n   - **박물관, 갤러리 방문**: 속초 주변에 있는 박물관이나 갤러리 탐방.\n\n4. **숙소에서 휴식**  \n   - 좋은 숙소를 예약했다면, 휴식과 함께 독서나 영화 감상 등 여유로운 시간을 보내는 것도 좋아요.\n\n---\n\n비 오는 날에도 안전하고 즐거운 여행 되시길 바랄게요! 필요하면 비 오는 날 맞춤 일정도 더 자세히 짜드릴 수 있어요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 350, 'prompt_tokens': 460, 'total_tokens': 810, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens'

In [13]:
summary = values["summary"]
summary

'사용자 김자윤님은 설악산으로 2박 3일 여행을 계획하고 있으며, 여행 스케줄을 요청했습니다. 이에 설악산 2박 3일 일정(권금성 케이블카, 대청봉 등반, 울산바위 트레킹, 속초 해산물 식사 등)을 제안해 드렸습니다. 이후 비가 올 경우 대처 방법과 대체 일정(설악 워터피아 온천, 속초 중앙시장, 아바이마을 방문 등)을 안내해 드렸습니다.'